In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [19]:
df = pd.read_csv('vehicle_insurance_claims.csv')
df.head()

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.91,0,466132,...,YES,71610,6510,13020,52080,Saab,92x,2004,Y,NaN
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.22,5000000,468176,...,?,5070,780,780,3510,Mercedes,E400,2007,Y,NaN
2,134,29,687698,2000-09-06,OH,100/300,2000,1413.14,5000000,430632,...,NO,34650,7700,3850,23100,Dodge,RAM,2007,N,NaN
3,256,41,227811,1990-05-25,IL,250/500,2000,1415.74,6000000,608117,...,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y,NaN
4,228,44,367455,2014-06-06,IL,500/1000,1000,1583.91,6000000,610706,...,NO,6500,1300,650,4550,Accura,RSX,2009,N,NaN


In [20]:
df.describe()

,months_as_customer,age,policy_number,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_year,_c39
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,0.0
mean,203.954000,38.948000,546238.648000,1136.000000,1256.406150,1.101000e+06,501214.488000,25126.100000,-26793.700000,11.644000,1.83900,0.992000,1.487000,52761.94000,7433.420000,7399.570000,37928.950000,2005.103000,NaN
std,115.113174,9.140287,257063.005276,611.864673,244.167395,2.297407e+06,71701.610941,27872.187708,28104.096686,6.951373,1.01888,0.820127,1.111335,26401.53319,4880.951853,4824.726179,18886.252893,6.015861,NaN
min,0.000000,19.000000,100804.000000,500.000000,433.330000,-1.000000e+06,430104.000000,0.000000,-111100.000000,0.000000,1.00000,0.000000,0.000000,100.00000,0.000000,0.000000,70.000000,1995.000000,NaN
25%,115.750000,32.000000,335980.250000,500.000000,1089.607500,0.000000e+00,448404.500000,0.000000,-51500.000000,6.000000,1.00000,0.000000,1.000000,41812.50000,4295.000000,4445.000000,30292.500000,2000.000000,NaN
50%,199.500000,38.000000,533135.000000,1000.000000,1257.200000,0.000000e+00,466445.500000,0.000000,-23250.000000,12.000000,1.00000,1.000000,1.000000,58055.00000,6775.000000,6750.000000,42100.000000,2005.000000,NaN
75%,276.250000,44.000000,759099.750000,2000.000000,1415.695000,0.000000e+00,603251.000000,51025.000000,0.000000,17.000000,3.00000,2.000000,2.000000,70592.50000,11305.000000,10885.000000,50822.500000,2010.000000,NaN
max,479.000000,64.000000,999435.000000,2000.000000,2047.590000,1.000000e+07,620962.000000,100500.000000,0.000000,23.000000,4.00000,2.000000,3.000000,114920.00000,21450.000000,23670.000000,79560.000000,2015.000000,NaN


In [53]:
zipcodes = pd.read_csv('uszips.csv')
for zip in df['insured_zip']:
    zip = int(str(zip)[1:])
    if zip in zipcodes['zip'].values:
        df['city'] = zipcodes[zipcodes['zip'] == zip]['city'].values[0]
        # print(df[df['insured_zip'] == zip]['policy_state'].values[0])

Hastings On Hudson, New York
Carolina, Puerto Rico
Sipesville, Pennsylvania
Jacksonville, Florida
Laredo, Missouri
Satin, Texas
Des Lacs, North Dakota
Roland, Arkansas
Stevensville, Pennsylvania
East Liverpool, Ohio
Hazlet, New Jersey
Jacksonville, Florida
Hiram, Georgia
Johnstown, Pennsylvania
Eagle Bend, Minnesota
Port Barre, Louisiana
Pinellas Park, Florida
Omaha, Nebraska
Sage, Arkansas
Boynton Beach, Florida
Reform, Alabama
Mosheim, Tennessee
Pierce, Nebraska
Lorimor, Iowa
Jacksboro, Texas
Olmito, Texas
Carp Lake, Michigan
Kennard, Texas
South Shore, Kentucky
Owanka, South Dakota
South Hero, Vermont
Saint Clair, Pennsylvania
Melbourne, Florida
Warren, Minnesota
Hopkinton, Massachusetts
Kenduskeag, Maine
Jamison, Pennsylvania
Cambridgeport, Vermont
Eldridge, Iowa
Levant, Kansas
Glen Rock, New Jersey
Chillicothe, Ohio
Madison, Alabama
Madison, Missouri
Numidia, Pennsylvania
Sparta, Wisconsin
Butler, Ohio
Picher, Oklahoma
Buffalo, Indiana
Batson, Texas
Crossville, Tennessee
Gulf Shor

KeyError: 'policy_state'